In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# For optimization using pygmo
import pygmo as pg
import numpy as np
from udp_initial_condition import udp_initial_condition

# For Plotting
import matplotlib.pyplot as plt

# For cProfile evaluation
import cProfile
import pstats

In [1]:
# Setup hyperparameters:
body_density = 533                  # https://sci.esa.int/web/rosetta/-/14615-comet-67p
target_altitude = 5000**2           # Target altitude squared
final_time = 20*3600.0              # Final time
start_time = 0                      # Starting time
time_step = 800              # Time step size for trajectory propagation.

# Bounds for initial variable domain: [r_x,r_y,r_z,v_x,v_y,v_z]
lower_bounds, upper_bounds = np.array([-6000,-6000,-6000,0,0,0]),np.array([6000,6000,6000,2.5,2.5,2.5])

# Optimization parameters
population_size = 10 #10
number_of_generations = 32 #32

NameError: name 'np' is not defined

In [5]:
# Main script:

def run():

    # Setup hyperparameters:
    body_density = 533                  # https://sci.esa.int/web/rosetta/-/14615-comet-67p
    target_altitude = 5000**2           # Target altitude squared
    final_time = 20*3600.0              # Final time
    start_time = 0                      # Starting time
    time_step = 800              # Time step size for trajectory propagation.

    # Bounds for initial variable domain: [r_x,r_y,r_z,v_x,v_y,v_z]
    lower_bounds, upper_bounds = np.array([-6000,-6000,-6000,0,0,0]),np.array([6000,6000,6000,2.5,2.5,2.5])

    # Optimization parameters
    population_size = 10
    number_of_generations = 32

    # Choose algorithm: 
    #  - "RKF78", "Euler" or "DP8713M"
    algorithm = "RKF78"

    # Setup problem class
    udp = udp_initial_condition(body_density, target_altitude, final_time, start_time, time_step, lower_bounds, upper_bounds, algorithm)

    # Setup user defined problem (udp)
    prob = pg.problem(udp)

    # Setup optimization algorithm
    algo = pg.algorithm(pg.sade(gen = number_of_generations))
    pop = pg.population(prob = prob, size = population_size)
    
    # Run optimization
    algo.set_verbosity(1)
    pop = algo.evolve(pop)

    # Logs for output
    print("Champion fitness value: ", pop.champion_f) 
    print("Champion chromosome: ", pop.champion_x) 

    # Retrieving champion chromosome
    x = pop.champion_x

    # Compute optimized trajectory
    fitness_value, trajectory_info  = udp.compute_trajectory(x)

    # Plot optimized trajectory
    udp.plot_trajectory(trajectory_info[0:3,:])

    plt.plot(trajectory_info[0,:],trajectory_info[1,:])

    return trajectory_info


def main():
    trajectory_info = run()


if __name__ == "__main__":
    cProfile.run("main()", "output.dat")

    with open("output_time.txt", "w") as f:
        p = pstats.Stats("output.dat", stream=f)
        p.sort_stats("time").print_stats()
    
    with open("output_calls.txt", "w") as f:
        p = pstats.Stats("output.dat", stream=f)
        p.sort_stats("calls").print_stats()


Physical dimension along x (UN-normalized):  5002.5703125 Km

   Gen:        Fevals:          Best:             F:            CR:            dx:            df:
      1             10    2.68538e+07        0.19635       0.211993        10202.6    1.20524e+10
      2             20    2.68538e+07        0.19635       0.211993        7144.54    1.14885e+10
      3             30    2.68538e+07        0.19635       0.211993        20314.1    1.09225e+10
      4             40    2.24456e+07       0.489608       0.407986        4975.88    9.48585e+09
      5             50    1.11203e+07       0.489608       0.407986        6106.45    9.46942e+09
      6             60    1.11203e+07       0.489608       0.407986        6106.45    9.46942e+09
      7             70    9.94289e+06        0.19635       0.211993         3389.8     5.5291e+09
      8             80    9.94289e+06        0.19635       0.211993        12354.1    4.17547e+09


KeyboardInterrupt: 

In [ ]:

# THIS IS FOR TESTING HEYOKA #

In [ ]:
# For optimization using pygmo
import pygmo as pg
import numpy as np
from udp_initial_condition import udp_initial_condition

# For Plotting
import matplotlib.pyplot as plt

In [17]:
# Setup hyperparameters:
body_density = 533                  # https://sci.esa.int/web/rosetta/-/14615-comet-67p
target_altitude = 5000**2           # Target altitude squared
final_time = 20*3600.0              # Final time
start_time = 0                      # Starting time
time_step = 800              # Time step size for trajectory propagation.

# Bounds for initial variable domain: [r_x,r_y,r_z,v_x,v_y,v_z]
lower_bounds, upper_bounds = np.array([-6000,-6000,-6000,0,0,0]),np.array([6000,6000,6000,2.5,2.5,2.5])

# Optimization parameters
population_size = 10
number_of_generations = 32

# Choose algorithm: 
#  - "RKF78", "Euler" or "DP8713M"
algorithm = "RKF78"

# Setup problem class
udp = udp_initial_condition(body_density, target_altitude, final_time, start_time, time_step, lower_bounds, upper_bounds, algorithm)


Physical dimension along x (UN-normalized):  5002.5703125 Km


In [18]:
# Compute optimized trajectory
x = [-1.06437413e+03, 4.33471220e+03, -3.29768568e+03, 2.67521306e-01, 3.42036498e-01, 3.66164107e-01]
fitness_value, trajectory_info  = udp.compute_trajectory(x)

TypeError: evaluate(): incompatible function arguments. The following argument types are supported:
    1. (vertices: List[List[float[3]]], faces: List[List[int[3]]], density: float, computation_point: List[float[3]]) -> Tuple[float, List[float[3]], List[float[6]]]
    2. (vertices: List[List[float[3]]], faces: List[List[int[3]]], density: float, computation_points: List[List[float[3]]]) -> List[Tuple[float, List[float[3]], List[float[6]]]]
    3. (input_files: List[str], density: float, computation_point: List[float[3]]) -> Tuple[float, List[float[3]], List[float[6]]]
    4. (input_files: List[str], density: float, computation_points: List[List[float[3]]]) -> List[Tuple[float, List[float[3]], List[float[6]]]]

Invoked with: array([[-1863.98704789,  2793.53966379,  -244.63722469],
       [-1840.52626511,  2925.80906249,   -58.07903007],
       [-1965.05644625,  2685.63458841,  -225.7048335 ],
       ...,
       [  852.47190008, -3211.33785321,   116.79048962],
       [  858.58453008, -

In [ ]:
# Plot optimized trajectory
udp.plot_trajectory(trajectory_info[0:3,:])

plt.plot(trajectory_info[0,:],trajectory_info[1,:])